# Extracting regional values from Met Office Global meteorological data

## Process
This notebook runs you through how to extract spatial mean values from gridded data using shapefiles. The process includes:

1. Load the Shapefile for the regions we want to subset with.
2. Determine the full lat-lon extent of the shapefile.
3. Use lookup table shapefile_attributes.json to determine what geometry attributes we want to use for this shapefile.
4. Use daterange to generate filenames of NetCDFs we want to load
5. Load gridded data from NetCDF files into memory using Iris (using [lazy loading](https://scitools.org.uk/iris/docs/latest/userguide/real_and_lazy_data.html)).
6. Subset the data to only the extent of the shapefile, improving the processing time.
7. Define the functions to be used in the pipeline.
8. Loop through all the regions in the shapefile subsetting, collapsing and generating a Pandas DataFrame for each region.
9. Each DataFrame is saved to CSV in a temporary location.
10. Collate all the region DataFrames into one DataFrame and save out to CSV
11. Delete the temporary files.

## Method
This process uses the polygon of a region (from the shapefile) to subset the gridded data by getting the **latitude-longitude bounding box** of the polygon, as described in this diagram:

<img src="images/coarse_spatial_mean_gridded.png" alt="Lat-Lon bounding box using polygon" style="height: 400px;"/> 

Each grid cell (small latitude-longitude box) contains a single value for a meteorological variable. The single value of that variable for the whole region/polygon is the mean of all the grid cell values in the bounding box i.e. lat-lon spatial mean.

For example, here we have air temperature values in a bounding box that covers the a polygon. The temperature value for the region is the mean value of the temperatures in the boundind box - 20.9°C.

<img src="images/spatial_mean_example.png" alt="The mean value for the temperature is 20.9°C" style="height: 400px;"/> 

#### Time
Of course we have ignored the time axis in this example, which is present in the gridded data but is handled for us by the Iris library as just another dimension. In this notebook we use daily data and will simply store the date for each value in the final tabular data.

#### Improvements
This process could be more accurate by only using the grid cells which actually overlap with the polygon and by weighting the grid cells according to how much of their area is within the polygon. Improvements like these are coming.

## Setup

In [ ]:
#Data
import iris
import cartopy.io.shapereader as shpreader
import pandas as pd
import cftime
import datetime

#Plotting
import cartopy
import cartopy.crs as ccrs
import iris.plot as iplt
import iris.quickplot as qplt
import matplotlib.pyplot as plt
%matplotlib inline

#System
import os
import sys
import glob
import json

#Met Office utils
import shape_utils as shape

#Supress warnings
import warnings
warnings.filterwarnings('ignore')

### 1. Load shapefile containing region polygons

In [ ]:
#Define shapefile path
SHAPEFILE = '../covid-19/it_shapefiles/gadm36_ITA_2.shp'
# SHAPEFILE = '../covid-19/uk_shapefiles/UK_covid_reporting_regions.shp'
# SHAPEFILE = '../covid-19/us_shapefiles/US_COUNTY_POP.shx'
# SHAPEFILE = '../covid-19/ug_shapefiles/gadm36_Uganda_2.shp'
# SHAPEFILE = '../covid-19/vt_shapefiles/gadm36_Vietnam_2.shp'
# SHAPEFILE = '../covid-19/br_shapefiles/gadm36_BRA_2.shp'

In [ ]:
#Get shapefile name
SHAPEFILE_NAME = SHAPEFILE.split('/')[-1].split('.')[-2]
SHAPEFILE_NAME

In [ ]:
#Load the shapefile
shape_reader = shpreader.Reader(SHAPEFILE)

In [ ]:
#How many regions are included?
len([record for record in shape_reader.records()])

In [ ]:
#Let's take a look at one
next(shape_reader.records())

In [ ]:
next(shape_reader.geometries())

## 2. Determine the full lat-lon extent of the shapefile

First we need some functions to help us

In [ ]:
def get_shapefile_extent(shape_reader, buffer=0, **kwargs):
    '''
    Get the extent (x1, x2, y1, y2) of the union of all the polygons
    in a shapefile reader object, with an optional buffer added on.
    
    Arguments:
        shape_reader (cartopy.io.shapereader): Shapefile reader object
        buffer (num): optional buffer to add to the extent of the shapefile
        **kwargs: Keyword arguments for shape_utils.Shape class
    
    Returns:
        extent (tuple): Extent float values in format (x1, x2, y1, y2)
        
    TODO: Split into
            - get_shapefile_shapelist(shape_reader)
            - get_shapelist_union(shapelist, **kwargs)
            - get_shape_extent(shape, buffer=0)
    '''
    #Get all the records from shapefile_reader
    recs = [rec for rec in shape_reader.records()]
    
    #Cycle through all the records in recs,
    #appending Shape object to ShapeList, if shape is valid
    shplist = shape.ShapeList([])
    for rec in recs:
        shp = shape.Shape(rec.geometry, rec.attributes, **kwargs) 
        if shp.is_valid:
            shplist.append(shp)
    
    #Get extent of union of all geometries
    wsen = shplist.unary_union().data.bounds
    
    #Rearrange extent from (x1, y1, x2, y2) to (x1, x2, y1, y2), adding buffer
    extent = (wsen[0]-buffer, wsen[2]+buffer, wsen[1]-buffer, wsen[3]+buffer)
    
    return extent

In [ ]:
def plot_extent(extent):
    '''
    Use Matplotlib to plot the given extent on a map of the world, 
    in Plate Carree projection, with coastlines and country boundaries.
    
    Arguments:
        extent (tuple): Float values for lat-lon coordinates in (x1, x2, y1, y2) format
    
    Returns: 
        Displays plot
    '''
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines('50m', color='b')
    ax.add_feature(cartopy.feature.BORDERS.with_scale('50m'), linestyle=':')
    ax.set_extent(extent)
    plt.show()

In [ ]:
%%time
#Get the extent of the shapefile, with a buffer of 1 degree
SHAPE_EXTENT = get_shapefile_extent(shape_reader, buffer=1)
SHAPE_EXTENT

In [ ]:
#Plot it onto a map of the world to check that it looks how we expect it to
plot_extent(SHAPE_EXTENT)

## 3. Use lookup table shapefile_attributes.json to determine what shapefile attributes we want to use later.

In [ ]:
#Define json path
json_read = './shapefile_attributes.json'

In [ ]:
#Load SHAPE_ATTRS from json file as a tuple
with open(json_read) as file:
    SHAPE_ATTRS = tuple(json.load(file)['shapefile_attributes'][SHAPEFILE_NAME])
    
SHAPE_ATTRS

In [ ]:
#The first attribute in SHAPE_ATTRS will be the shapefile attribute we iterate over in our pipeline
SHAPE_IDS = tuple(record.attributes[SHAPE_ATTRS[0]] for record in shape_reader.records())
SHAPE_IDS[0:10]

## 4. Use daterange to generate filenames of NetCDFs we want to load

In [ ]:
#Define date range
start = datetime.date(2020, 3, 1)
stop = datetime.date(2020, 3, 11)
step = datetime.timedelta(days=1)
DATERANGE = pd.date_range(start, stop, freq=step)
DATERANGE

In [ ]:
filename_fmt = f'*_%Y%m%d.nc'
FILENAMES = list(DATERANGE.strftime(filename_fmt))
print(FILENAMES[0])
print(FILENAMES[-1])

## 5. Load gridded data from NetCDF files into memory using Iris (using [lazy loading](https://scitools.org.uk/iris/docs/latest/userguide/real_and_lazy_data.html)).

The files for each variable are contained in a separate folder.

In [ ]:
%%time
#List all the filepaths and store in a dict with each variable as a key
folder = '/data/misc/covid-19/data_nc_daily/'
filepaths = {}
for path in os.listdir(folder):
    filepaths[path] = []
    for filename in FILENAMES:
        filepaths[path].extend(glob.glob(os.path.join(folder, path, filename)))
variables = list(filepaths.keys())

print(variables)
print(f'Number of files for each variable: {len(filepaths[variables[0]])}')

In [ ]:
%%time
#Run through all the variables and append the loaded cubes to a CubeList
cubes = iris.cube.CubeList([])

for var in variables:
    cubes.extend(iris.load(filepaths[var]))
    
print(cubes)

## 6. Subset global data based on the extent of the shapefile

In [ ]:
#Define CoordExtent objects for x and y axes using SHAPE_EXTENT
x_axis = cubes[0].coord(axis='x')
y_axis = cubes[0].coord(axis='y')

x_extent = iris.coords.CoordExtent(x_axis, SHAPE_EXTENT[0], SHAPE_EXTENT[1])
y_extent = iris.coords.CoordExtent(y_axis, SHAPE_EXTENT[2], SHAPE_EXTENT[3])

In [ ]:
#Subset cubes
shape_cubes = iris.cube.CubeList([cube.intersection(x_extent, y_extent) for cube in cubes])
print(shape_cubes)

In [ ]:
#Plot the first time step of the first cube in cubes
#To check that we've subsetted correctly
qplt.contourf(shape_cubes[0][0], cmap='Purples')
plt.gca().coastlines('50m', color='blue')
plt.gca().add_feature(cartopy.feature.BORDERS.with_scale('50m'), linestyle=':')

In [ ]:
#Extract the coordinate reference system from one of the cubes. We will use this later.
CRS = shape_cubes[0].coord_system()
CRS

## 7. Define the functions to be used in the pipeline.

### Shapefile functions

In [ ]:
def get_shape_record(target, shape_reader=shape_reader, attribute=SHAPE_ATTRS[0]):
    '''
    Get a record from the shape_reader with a target attribute.
    
    '''
    result = None
    for record in shape_reader.records():
        shape_id = record.attributes[attribute]
        if shape_id == target:
            result = record
            break
    if result is None:
        emsg = f'Could not find record with {attribute} = "{target}".'
        raise ValueError(emsg)
    return result

In [ ]:
#Create a random ID generator
from random import randint
def rand_id(ids=SHAPE_IDS):
    '''
    Return a random id
    Useful for testing
    '''
    rand_i = randint(0, len(ids)-1)
    return ids[rand_i]

In [ ]:
#Get a random geometry to check it's all working as expected
i = rand_id()
print(i)
get_shape_record(i).geometry

### Gridded data functions

In [ ]:
def get_cell_method(cube, coord='time'):
    '''
    Get the cell method with coord in coord_names
    '''
    result = None
    for method in cube.cell_methods:
        if coord in method.coord_names:
            result = method.method
            break
    
    return result

In [ ]:
def parse_data_name(cube):
    '''
    Parse the name, cell methods and units in a cube to return a column name
    To be used in a Pandas DataFrame
    '''
    name = cube.name()
    time_method = get_cell_method(cube, 'time').replace('imum', '')
    space_method = get_cell_method(cube, 'longitude')
    units = cube.units
    
    if name == 'm01s01i202':
        name = 'short_wave_radiation'
        if space_method.startswith('var'):
            units = 'W2 m-4'
        else:
            units = 'W m-2'
    
    if space_method:
        result = f'{name}_{time_method}_{space_method.replace("imum", "")} ({units})'
    else:
        result = f'{name}_{time_method} ({units})'
    
    return result

In [ ]:
def get_date(dt):
    '''
    Return date from datetime-like object dt
    '''
    return datetime.date(dt.year, dt.month, dt.day)

In [ ]:
def get_column_order(start, end):
    '''
    
    '''
    starts = tuple(start)
    
    ends = tuple(sorted([c for c in end if c not in starts]))
    
    return starts+ends

In [ ]:
def extract_collapse_df(shape_id, cubes=shape_cubes, **kwargs):
    '''
    Extract subcubes from cubes using geometry of shape_id
    Collapse the cube acros x and y coords to get the MEAN and VARIANCE
    Collect data in a dataframe for this shape_id
    
    Extract method:
        Extracts XY bounding box around geometry
        Refer to [Method](#Method) at top of this notebook for detailed description
        
    Arguments:
        shape_id (str): ID of geometry used for subsetting
        cubes (iris.CubeList): List of Iris cubes to be subsetted
        
    Returns: 
        df (pandas.DataFrame): DataFrame containing shape_id attributes 
                               and MEAN+VARIANCE of data in cubes for shape_id geometry
    '''
    #Create a Shape object from the record for shape_id
    region = get_shape_record(shape_id, **kwargs)
    shp = shape.Shape(region.geometry, region.attributes, coord_system=CRS)
    
    #Extract sub_cubes from cubes using shp
    sub_cubes = shp.extract_subcubes(cubes)
    
    #Collapse cubes across x and y coords with to get mean and variance
    mean_cubes = [cube.collapsed([cube.coord(axis='x'),cube.coord(axis='y')], iris.analysis.MEAN) for cube in sub_cubes]
    var_cubes = [cube.collapsed([cube.coord(axis='x'),cube.coord(axis='y')], iris.analysis.VARIANCE) for cube in sub_cubes]
    
    #Line up data and column names for Pandas DataFrame
    time = mean_cubes[0].coord('time')
    length = len(time.points)
    data = {'shapefile': [SHAPEFILE_NAME]*length}
    data.update({name: [region.attributes[name]]*length for name in SHAPE_ATTRS})
    data.update({'date': [get_date(cell.point) for cell in time.cells()]})
    data.update({parse_data_name(cube): cube.data for cube in mean_cubes})
    data.update({parse_data_name(cube): cube.data for cube in var_cubes})
    
    #Get a column order so that all dataframes have the same column order
    column_order = get_column_order(['shapefile']+list(SHAPE_ATTRS)+['date'], end=data.keys())
    
    #Create DataFrame
    df = pd.DataFrame(data, columns=column_order)

    return df

In [ ]:
%%time
cblst = shape_cubes
extract_collapse_df(rand_id(), cblst)

## 8. Loop through all the regions in the shapefile; subsetting, collapsing, and generating a Pandas DataFrame<br>9. Each DataFrame is written to CSV

In [ ]:
#Specify the folder and part of the filename for the csv files to be written to
CSV_FOLDER = f"/data/share/kaedonkers/{SHAPEFILE_NAME.replace('_', '')}_region_csvs"
CSV_NAME = 'metoffice_global_daily'

print(CSV_FOLDER)

In [ ]:
#Create CSV_FOLDER if it doesn not exist
if not os.path.isdir(CSV_FOLDER):
    os.mkdir(CSV_FOLDER)

In [ ]:
#List the csvs already written in CSV_FOLDER
CSVS_WRITTEN = glob.glob(os.path.join(CSV_FOLDER, '*.csv'))
len(CSVS_WRITTEN)

In [ ]:
def get_csv_name(folder, shapefile, shape_id, data_name, start_dt, end_dt, separator='_', dtfmt='%Y%m%d'):
    '''
    
    '''
    extension = '.csv'
    
    #Cut out underscores from shapefile name to reduce overall length
    shapefile = shapefile.replace('_', '')
    
    #Format the daterange that be at the end of the filename
    dt_range = f"{start_dt.strftime(dtfmt)}-{end_dt.strftime(dtfmt)}"
    
    #Join all the filename parts with separator
    if shape_id:
        shape_id = shape_id.replace('_', '-').replace('.', '-')
        filename = separator.join([shapefile, shape_id, data_name, dt_range])
    else:
        filename = separator.join([shapefile, data_name, dt_range])
    
    #Join filename with folder
    filepath = os.path.join(folder, f"{filename}{extension}")
    
    return filepath

In [ ]:
#Test get_csv_name
get_csv_name(CSV_FOLDER, SHAPEFILE_NAME, rand_id(), CSV_NAME, DATERANGE[0], DATERANGE[-1])

#### Loop through all shape geometries

In [ ]:
%%time
#This will loop through all the region IDs, executing extract_collapse_df for each region and saving it to a CSV file
#Any errors will be caught and printed, but the loop will continue onto the next ID
#Files are writen to CSV_FOLDER

start = len(CSVS_WRITTEN)
# stop = len(region_ids)
stop = 3
for shape_id in SHAPE_IDS[start:stop]:
    try:
        df = extract_collapse_df(shape_id, shape_cubes)
        fname = get_csv_name(CSV_FOLDER, SHAPEFILE_NAME, shape_id, CSV_NAME, DATERANGE[0], DATERANGE[-1])
        df.to_csv(fname, index=False)
        print(f'  [{shape_id}] {fname}: Success')
    except Exception as e:
        print(f'x [{shape_id}] {fname}: Error \n  x  {e}')

## 10. Load all the region CSVs, collate into one large DataFrame and save out to CSV.

In [ ]:
#List the csvs written in CSV_FOLDER
CSVS_READ = glob.glob(os.path.join(CSV_FOLDER, '*.csv'))
len(CSVS_READ)

In [ ]:
%%time
#Now load all the CSVs for each region and combine into one large dataframe
df = pd.concat([pd.read_csv(csv) for csv in CSVS_READ], ignore_index=True)
df

In [ ]:
#Write the collated csv to the parent folder of the regional csvs and generate a filepath
CSV_COLLATE_FOLDER = '/'.join(CSV_FOLDER.split('/')[0:-1])
print(CSV_FOLDER)
print(CSV_COLLATE_FOLDER)

CSV_COLLATE_FILEPATH = get_csv_name(CSV_COLLATE_FOLDER, SHAPEFILE_NAME, None, CSV_NAME, DATERANGE[0], DATERANGE[-1])
print(CSV_COLLATE_FILEPATH)

In [ ]:
#And save to a CSV
df.to_csv(CSV_COLLATE_FILEPATH, index=False)

In [ ]:
#We can read it back in to check that it wrote correctly
pd.read_csv(CSV_COLLATE_FILEPATH)

## 11. Delete temporary files

In [ ]:
#List the CSV_FOLDER
!ls -alh {CSV_FOLDER}

In [ ]:
#Remove CSV_FOLDER
!rm -rd {CSV_FOLDER}

In [ ]:
#List CSV_FOLDER again, to check everything is gone
!ls -alh {CSV_FOLDER}